# Chapter 2 : Modeling in the Frequency Domain (Part 3)
---

In [1]:
import sys

if (path := "C:/Users/Tom/pycharm-projects/python-control") not in sys.path:
    sys.path.append(path)

import sympy as sp
from IPython.display import display

from python_control import Quantity, s, TransferFunction

from python_control.modeling.electrical import (
    Resistor, DCMotor,
    Mesh, Circuit
)

from python_control.modeling.mechanical import (
    Inertia, TorsionDamper, 
    GearRatio,
    Mass, Spring, Damper
)

Q_ = Quantity

## 2.8 : Electromechanical System Transfer Functions

### Example 2.23 : Transfer Function - DC Motor and Load

Given the dc-motor-gearbox-load system and the torque-speed curve of the dc-motor, find the transfer function $\theta_L(s) / E_a(s)$ of this system.

Motor specifications:
- The armature (rotor) inertia of the motor: 5 kg.m²
- The viscous damping coefficient of the rotor: 2 N.m.s/rad
- Stall torque @ 100 V: 500 N.m
- No-load speed @ 100 V: 50 rad/s

Load specifications:
- Gearbox ratio: 1:10
- Load inertia: 700 kg.m²
- Viscous damping coefficient: 800 N.m.s/rad 

Note that the armature resistance $R_a$ has not been specified, and it cannot be determined directly from the torque-speed curve. However, this is not a real issue here, as it is actually only the ratio $K_t / R_a$ that we need to know to model the dc-motor and this ratio can be deduced from the given torque-speed curve: $K_t / R_a = T_{stall} / E_a$. In fact, we can replace (in our mind) the ratio $K_t / R_a$ by $K_t$ (as if we assume that $R_a$ = 1 $\ohm$).

First, we determine the overall mechanical impedance on the motor shaft. For this, we need to reflect the load impedances to the motor shaft.

In [2]:
Z_Ja = Inertia(Q_(5, 'kg * m**2')).Z
Z_Da = TorsionDamper(Q_(2, 'N * m * s / rad')).Z

Z_Jl = Inertia(Q_(700, 'kg * m**2')).Z
Z_Dl = TorsionDamper(Q_(800, 'N * m * s / rad')).Z

gear_ratio = GearRatio(N_in=100, N_out=1000)

Z_Jl_to_a = gear_ratio.reflect_to_input(Z_out=Z_Jl)
Z_Dl_to_a = gear_ratio.reflect_to_input(Z_out=Z_Dl)

Z_Jm = Z_Ja + Z_Jl_to_a
Z_Dm = Z_Da + Z_Dl_to_a

In [3]:
Z_Jm.expr

12.0*s**2

In [4]:
Z_Dm.expr

10.0*s

We determine the motor constants from the torque-speed curve:

In [5]:
T_stall = Q_(500, 'N * m')
E_a = Q_(100, 'V')
omega_nl = Q_(50, 'rad/s')

Kt = T_stall / E_a  # actually Kt / Ra
Kb = E_a / omega_nl

In [6]:
Kt

<Quantity(5.0, 'newton * meter / volt')>

In [7]:
Kb

<Quantity(2.0, 'volt * second / radian')>

Now, we can model the dc-motor with the attached load. For this, we use the `DCMotor` class. 

In [8]:
dc_motor = DCMotor(
    Z_Ra=Resistor(Q_(1, 'ohm')).Z,  # see our assumption above
    Z_Jm=Z_Jm,
    Z_Dm=Z_Dm,
    Kb=Kb,
    Kt=Kt
)

To get at the transfer function we are looking for, we first call the method `trf_fun_angle_voltage` on our `dc_motor` instance. This will return us the transfer function $\theta_m(s) / E_a(s)$, with $\theta_m$ the rotation angle of the motor shaft. However, we want the transfer function $\theta_L(s) / E_a(s)$, with $\theta_L$ the rotation angle of the gearbox output shaft.

In [9]:
G = dc_motor.trf_fun_angle_voltage()
G.expr

0.416666666666667/(1.0*s**2 + 1.66666666666667*s)

To get at the desired transfer function, we need to multiply `G` with the angle ratio of the gearbox $\theta_2 / \theta_1 = N_1 / N_2$.

In [10]:
G = G * gear_ratio.angle_ratio
G.expr

0.0416666666666667/(1.0*s**2 + 1.66666666666667*s)

## 2.9 : Electric Circuit Analogs

### Example 2.24 : Converting a Mechanical System to a Series Analog

Repeat example 2.17 using mesh analysis from electrical network theory.

![two-degrees-of-freedom translational mechanical system](./images/example_2-17.png)

**Rules**
- Impedances connected with a mass form a mesh or loop, where impedances between two masses are common to the two loops.
- The mass velocity in the mechanical system is analog to current in the corresponding mesh of the electrical network.
- An external force applied to a mass in the mechanical system is analog to a voltage source in the corresponding mesh of the electrical network.
- A mechanical spring is analog to a electrical capacitor with $C = 1/K$.
- A mechanical mass is analog to an electrical inductor with $L = M$.
- A mechanical damper is analog to an electrical resistor with $R = D$.

The two-degree-of-freedom translational mechanical system has two masses which can move independently from each other. Mass $M_1$ has velocity $v_1(t)$, which is analog to a mesh current in loop 1. Mass $M_2$ has velocity $v_2(t)$, which is analog to a mesh current in loop 2. The force $f(t)$, applied to mass $M_1$, is analog to a voltage source in loop 1.

![series analog of mechanical system](./images/example_2-24.png)

Although we could represent all the mechanical components in the system by their electrical counterparts, this is not necessary when performing a mesh analysis using classes `Mesh` and `Circuit`. When, instead of an electrical component, a mechanical component is added to a mesh, its "velocity impedance" $F(s) / V(s)$ will be considered instead of its conventional "displacement impedance" $F(s) / X(s)$.

In [11]:
F = sp.Symbol('F')

K1 = Spring('K1')
M1 = Mass('M1')
D1 = Damper('f_v1')
D3 = Damper('f_v3')
K2 = Spring('K2')
D2 = Damper('f_v2')
M2 = Mass('M2')
K3 = Spring('K3')

mesh1 = Mesh('1')
mesh1.add_voltage_source(F)
mesh1.add_component(K1)
mesh1.add_component(M1)
mesh1.add_component(D1)
mesh1.add_component(D3)
mesh1.add_component(K2)

mesh2 = Mesh('2')
mesh2.add_component(D3)
mesh2.add_component(K2)
mesh2.add_component(M2)
mesh2.add_component(D2)
mesh2.add_component(K3)

circuit = Circuit()
circuit.add_mesh(mesh1)
circuit.add_mesh(mesh2)

sol = circuit.solve()

In [12]:
for eq in circuit.equations:
    display(eq)

Eq(V_1*(1.0*K1/s + 1.0*K2/s + 1.0*M1*s + 1.0*f_v1 + 1.0*f_v3) + V_2*(-1.0*K2/s - 1.0*f_v3), F)

Eq(V_1*(-1.0*K2/s - 1.0*f_v3) + V_2*(1.0*K2/s + 1.0*K3/s + 1.0*M2*s + 1.0*f_v2 + 1.0*f_v3), 0)

These equations now relate the velocities of masses $M_1$ and $M_2$ to the external forces, instead of their displacements. In example 2.17 the transfer function $X_2(s) / F(s)$ for the system is asked. We can get at the transfer function $G(s) = V_2(s) / F(s)$ from the solution of the circuit. As $V(s) = s \cdot X(s)$, we can finally arrive at the desired transfer function $X_2(s) / F(s)$ by multiplying $G(s)$ with $1/s$.

In [13]:
V2 = sol[mesh2.current]
F = mesh1.voltage_sources[0]
G = TransferFunction(V2 / F) * (1 / s)
G.expr

(K2/(M1*M2) + f_v3*s/(M1*M2))/(s**4 + s**3*(M1*f_v2 + M1*f_v3 + M2*f_v1 + M2*f_v3)/(M1*M2) + s**2*(K1*M2 + K2*M1 + K2*M2 + K3*M1 + f_v1*f_v2 + f_v1*f_v3 + f_v2*f_v3)/(M1*M2) + s*(K1*f_v2 + K1*f_v3 + K2*f_v1 + K2*f_v2 + K3*f_v1 + K3*f_v3)/(M1*M2) + (K1*K2 + K1*K3 + K2*K3)/(M1*M2))